# Tool

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 定义工具

组成工具的组件：

- name (str), **必要字段** 在智能体的可选工具集中必须是唯一名称
- description (str), **可选字段** 强烈建议提供，智能体可以通过这个字段正确地找到工具
- args_schema (Pydantic BaseModel), **可选字段** 强烈建议提供，可用于提供数据样本或参数校验

In [2]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

### `@tool` 装饰器

如果想在定义 Python 函数的同时将其声明为工具，使用 `@tool` 是最简单的方案。

In [3]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [4]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


### `@tool` + Pydantic 修饰

使用 Pydantic 可以丰富工具的说明，这非常有利于智能体的推理，因此是强烈建议的。

In [5]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [6]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
search-tool(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}
True


### BaseTool 子类

实际上，`@tool`装饰器是 BaseTool 的一个简化版本。

In [14]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return "LangChain"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [8]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

custom_search
useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


### StructuredTool 子类

你也可以将已经定义好的 Python 函数转换为 Tool 给智能体使用。

In [9]:
def search_function(query: str):
    return "LangChain"

search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

In [10]:
print(search.name)
print(search.description)
print(search.args)

Search
Search(query: str) - useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


### StructuredTool + Pydantic 修饰

In [12]:
def search_function(query: str):
    return "LangChain"

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
    
search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    args_schema=SearchInput,
    return_direct=True
)

In [13]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

Search
Search(query: str) - useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}
True
